New logic: Swire would want to predict the abandonment couple of days before the end of the window. 
Fit training data with cart abandonment but only up to the last 2 windows of puchase or abandon. 
Then for the test data, test/predict only couples days in not wait untill the end of the window. 

In [2]:
import pandas as pd
import numpy as np
# import os
# from sklearn.model_selection import train_test_split
# import statsmodels.api as sm
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
# import statsmodels.formula.api as smf
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import StratifiedKFold, cross_validate
# from lightgbm import LGBMClassifier
# import xgboost as xgb
# from catboost import CatBoostClassifier
# #import duckdb

# Clean google analytics.csv and join with customer.csv 


In [3]:
ga = pd.read_csv("Copy of google_analytics.csv")
pd.set_option('display.max_columns', None)
ga.head()

,CUSTOMER_ID,EVENT_DATE,EVENT_TIMESTAMP,EVENT_NAME,DEVICE_CATEGORY,DEVICE_MOBILE_BRAND_NAME,DEVICE_OPERATING_SYSTEM,EVENT_PAGE_NAME,EVENT_PAGE_TITLE,ITEMS
0,501508595,2025-04-07,2025-04-07T14:57:40.713Z,view_search_results,mobile,Samsung,Android,Product List,NaN,[]
1,501730915,2025-04-07,2025-04-07T19:28:54.546Z,view_item_list,desktop,Google,Windows,NaN,Search,"[{""item_id"":""158520"",""quantity"":""1""},{""item_id..."
2,501567329,2025-04-07,2025-04-07T21:40:56.281Z,purchase,desktop,Google,Windows,MyCoke Orders - Purchase Success,Order,"[{""item_id"":""138335"",""quantity"":""2""},{""item_id..."
3,501765083,2025-04-08,2025-04-08T06:06:36.717Z,user_engagement,mobile,Apple,iOS,NaN,NaN,[]
4,501707837,2025-04-08,2025-04-08T18:12:10.511Z,view_item_list,desktop,Google,Windows,MyCoke Orders,Category: All Products,"[{""item_id"":""412588"",""quantity"":""1""},{""item_id..."


In [4]:
# Getting only cart affected events
ga_cart_events = ga[ga['EVENT_NAME'].isin(['add_to_cart', 'remove_from_cart','purchase'])]

In [5]:
# Join customer.csv with ga_cart_events to get customer's local time 
customer = pd.read_csv('Copy of customer.csv')

customer_ga_cart_events = pd.merge(ga_cart_events, customer, left_on='CUSTOMER_ID', right_on='CUSTOMER_NUMBER', how='left')

In [6]:
from zoneinfo import ZoneInfo
import pandas as pd

# ---- 1) City → timezone map (as you had) ----
city_tz = {
    "Tacoma, WA":"America/Los_Angeles","Spokane, WA":"America/Los_Angeles",
    "Arlington, WA":"America/Los_Angeles","Bellevue, WA":"America/Los_Angeles",
    "Walla Walla, WA":"America/Los_Angeles","Reno, NV":"America/Los_Angeles",
    "Elko, NV":"America/Los_Angeles","Pendleton, OR":"America/Los_Angeles",
    "Eugene, OR":"America/Los_Angeles","Wilsonville, OR":"America/Los_Angeles",
    "Bend, OR":"America/Los_Angeles","LaGrande, OR":"America/Los_Angeles",
    "Lewiston, ID":"America/Los_Angeles","Wenatchee, WA":"America/Los_Angeles",
    "Bremerton, WA":"America/Los_Angeles",
    "Denver, CO":"America/Denver","Colorado Springs, CO":"America/Denver",
    "Grand Junction, CO":"America/Denver","Glenwood Springs, CO":"America/Denver",
    "Pueblo, CO":"America/Denver","Alamosa, CO":"America/Denver",
    "Johnstown, CO":"America/Denver","Cheyenne, WY":"America/Denver",
    "Richfield, UT":"America/Denver","Draper, UT":"America/Denver",
    "Logan, UT":"America/Denver","Ogden, UT":"America/Denver",
    "Price, UT":"America/Denver","Albuquerque, NM":"America/Denver",
    "Scottsbluff, NE":"America/Denver",
    "Boise, ID":"America/Boise","Idaho Falls, ID":"America/Boise","Pocatello, ID":"America/Boise",
    "Tempe, AZ":"America/Phoenix","Tucson, AZ":"America/Phoenix",
    "Prescott, AZ":"America/Phoenix","Glendale, AZ":"America/Phoenix",
    "Kingman, AZ":"America/Phoenix","Huachuca City, AZ":"America/Phoenix",
    "Show Low, AZ":"America/Phoenix","Flagstaff, AZ":"America/Phoenix",
    "Yuma, AZ":"America/Phoenix",
}

# ---- 2) Parse timestamp as UTC (kept your column name) ----
customer_ga_cart_events["EVENT_TIMESTAMP_UTC"] = pd.to_datetime(
    customer_ga_cart_events["EVENT_TIMESTAMP"], utc=True, errors="coerce"
)

# ---- 3) Map timezone from CITY (use CITY consistently) ----
customer_ga_cart_events["TIMEZONE"] = customer_ga_cart_events["SALES_OFFICE_DESCRIPTION"].map(city_tz)

# ---- 4) Convert to local datetime per-row (only when TIMEZONE present) ----
customer_ga_cart_events["EVENT_LOCAL_DT"] = customer_ga_cart_events.apply(
    lambda row: (
        row["EVENT_TIMESTAMP_UTC"].astimezone(ZoneInfo(row["TIMEZONE"]))
        if pd.notna(row["TIMEZONE"]) and pd.notna(row["EVENT_TIMESTAMP_UTC"])
        else pd.NaT
    ),
    axis=1
)

# ---- 5) Convenience columns: date / hour / day-of-week ----
customer_ga_cart_events["EVENT_LOCAL_DATE"] = customer_ga_cart_events["EVENT_LOCAL_DT"].apply(
    lambda v: v.date() if pd.notna(v) else pd.NaT
)
customer_ga_cart_events["EVENT_LOCAL_TIME"] = customer_ga_cart_events["EVENT_LOCAL_DT"].apply(
    lambda v: v.strftime("%I:%M:%S %p") if pd.notna(v) else pd.NA
)
customer_ga_cart_events["EVENT_LOCAL_DOW"] = customer_ga_cart_events["EVENT_LOCAL_DT"].apply(
    lambda v: v.day_name() if pd.notna(v) else pd.NA
)

# ---- 6) Drop duplicates (use the actual UTC column name) ----
customer_ga_cart_events = customer_ga_cart_events.drop_duplicates(
    subset=["CUSTOMER_ID","EVENT_TIMESTAMP_UTC","EVENT_NAME"],
    keep="first"
)

# ---- 7) Select relevant columns (use consistent names) ----
cols = [
    "CUSTOMER_ID","CITY","TIMEZONE","EVENT_NAME",
    "EVENT_LOCAL_TIME","EVENT_LOCAL_DT","EVENT_LOCAL_DATE",
    "EVENT_LOCAL_HOUR","EVENT_LOCAL_DOW",
    "DEVICE_CATEGORY","DEVICE_MOBILE_BRAND_NAME","DEVICE_OPERATING_SYSTEM",
    "EVENT_PAGE_NAME","EVENT_PAGE_TITLE"
]
existing_cols = [c for c in cols if c in customer_ga_cart_events.columns]
customer_ga_cart_events = customer_ga_cart_events[existing_cols]

customer_ga_cart_events.head()


,CUSTOMER_ID,TIMEZONE,EVENT_NAME,EVENT_LOCAL_TIME,EVENT_LOCAL_DT,EVENT_LOCAL_DATE,EVENT_LOCAL_DOW,DEVICE_CATEGORY,DEVICE_MOBILE_BRAND_NAME,DEVICE_OPERATING_SYSTEM,EVENT_PAGE_NAME,EVENT_PAGE_TITLE
0,501567329,America/Los_Angeles,purchase,02:40:56 PM,2025-04-07 14:40:56.281000-07:00,2025-04-07,Monday,desktop,Google,Windows,MyCoke Orders - Purchase Success,Order
1,501702910,America/Denver,add_to_cart,12:30:14 PM,2025-05-24 12:30:14.025000-06:00,2025-05-24,Saturday,desktop,Google,Windows,NaN,Search
2,501565777,America/Phoenix,add_to_cart,01:08:07 PM,2025-05-20 13:08:07.167000-07:00,2025-05-20,Tuesday,mobile,Samsung,Android,NaN,Search
3,501752380,America/Los_Angeles,add_to_cart,02:25:29 PM,2025-05-16 14:25:29.494000-07:00,2025-05-16,Friday,desktop,Apple,Macintosh,NaN,Search
4,500549947,America/Los_Angeles,add_to_cart,06:12:45 PM,2025-04-07 18:12:45.609000-07:00,2025-04-07,Monday,mobile,Apple,iOS,Product List,NaN


# The windows from master.csv

In [7]:
windows = pd.read_csv('master.csv')

In [8]:
pd.set_option('display.max_columns', None)

windows.head()

,Unnamed: 0,CUSTOMER_ID,ANCHOR_DATE,SALES_OFFICE_x,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC,CUTOFFTIME__C,NEXT_ANCHOR_SAME_WD
0,219,500245685,2024-06-10,G111,48 Hours,Monday,7,OFS,5:00:00 PM,2024-06-17
1,220,500245685,2024-06-17,G111,48 Hours,Monday,7,OFS,5:00:00 PM,2024-06-24
2,221,500245685,2024-06-24,G111,48 Hours,Monday,7,OFS,5:00:00 PM,2024-07-01
3,222,500245685,2024-07-01,G111,48 Hours,Monday,7,OFS,5:00:00 PM,2024-07-08
4,223,500245685,2024-07-08,G111,48 Hours,Monday,7,OFS,5:00:00 PM,2024-07-15


# windows join customer_ga_cart_events

In [10]:
# Ensure datetime types
windows["ANCHOR_DATE"] = pd.to_datetime(windows["ANCHOR_DATE"])
windows["NEXT_ANCHOR_SAME_WD"] = pd.to_datetime(windows["NEXT_ANCHOR_SAME_WD"])
customer_ga_cart_events["EVENT_LOCAL_DATE"] = pd.to_datetime(customer_ga_cart_events["EVENT_LOCAL_DATE"])

# Perform a LEFT merge (windows = left table)
windows_with_ga = pd.merge(
    windows,
    customer_ga_cart_events,
    on="CUSTOMER_ID",
    how="left"
)


windows_with_ga["WINDOW_START_LOCAL"] = pd.to_datetime(
    windows_with_ga["ANCHOR_DATE"].dt.strftime("%Y-%m-%d") + " " + windows_with_ga['CUTOFFTIME__C'],   #CUTOFFTIME__C	?
    errors="coerce"
)
windows_with_ga["WINDOW_END_LOCAL"] = pd.to_datetime(
    windows_with_ga["NEXT_ANCHOR_SAME_WD"].dt.strftime("%Y-%m-%d") + " " + windows_with_ga['CUTOFFTIME__C'],
    errors="coerce"
)

# 2) Drop timezone from the event’s local datetime so it’s naive too
windows_with_ga["EVENT_LOCAL_NAIVE"] = windows_with_ga["EVENT_LOCAL_DT"].apply(
    lambda v: pd.NaT if pd.isna(v) else pd.Timestamp(v).tz_localize(None)
)

# 1) Build in-window mask (compare naive-to-naive)
in_window = (
    (windows_with_ga["EVENT_LOCAL_NAIVE"] >= windows_with_ga["WINDOW_START_LOCAL"]) &
    (windows_with_ga["EVENT_LOCAL_NAIVE"] <  windows_with_ga["WINDOW_END_LOCAL"])
)

# 2) Keep ALL windows; just mark which rows are in-window
windows_with_ga["IN_WINDOW"] = in_window

# 3) Null out event fields for rows outside the window so they won't count in aggregates
event_cols = [
    "EVENT_NAME","EVENT_LOCAL_DT","EVENT_LOCAL_DATE","EVENT_LOCAL_TIME",
    "DEVICE_CATEGORY","DEVICE_MOBILE_BRAND_NAME","DEVICE_OPERATING_SYSTEM",
    "EVENT_PAGE_NAME","EVENT_PAGE_TITLE"
]
windows_with_ga.loc[~in_window, event_cols] = pd.NA

# 4) Optional indicators for counts
windows_with_ga["is_add_to_cart"]       = (windows_with_ga["EVENT_NAME"] == "add_to_cart").astype("Int64")
windows_with_ga["is_remove_from_cart"]  = (windows_with_ga["EVENT_NAME"] == "remove_from_cart").astype("Int64")
windows_with_ga["is_purchase"]          = (windows_with_ga["EVENT_NAME"] == "purchase").astype("Int64")



/var/folders/hd/fbf93pws5pn1ycb22tw5k1240000gn/T/ipykernel_63557/1821675566.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  windows_with_ga["WINDOW_START_LOCAL"] = pd.to_datetime(
/var/folders/hd/fbf93pws5pn1ycb22tw5k1240000gn/T/ipykernel_63557/1821675566.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  windows_with_ga["WINDOW_END_LOCAL"] = pd.to_datetime(


In [11]:
windows_with_ga_1 = (
    windows_with_ga
    .groupby(["CUSTOMER_ID","WINDOW_START_LOCAL","WINDOW_END_LOCAL"], as_index=False)
    .agg(
        SALES_OFFICE=("SALES_OFFICE_x","first"),
        SHIPPING_CONDITIONS_DESC=("SHIPPING_CONDITIONS_DESC","first"),
        WEEK_DAY_OF_ANCHOR_DATE=("WEEK_DAY_OF_ANCHOR_DATE","first"),
        WINDOW_FREQUENCY=("WINDOW_FREQUENCY","first"),
        DISTRIBUTION_MODE_DESC=("DISTRIBUTION_MODE_DESC","first"),
        TIMEZONE=("TIMEZONE","first"),

        add_to_cart_count=("is_add_to_cart","sum"),
        remove_from_cart_count=("is_remove_from_cart","sum"),
        purchase_count=("is_purchase","sum"),
        has_any_event=("IN_WINDOW","any"),
        first_event_time=("EVENT_LOCAL_NAIVE","min"),
        last_event_time=("EVENT_LOCAL_NAIVE","max"),
    )
).fillna({"add_to_cart_count":0,"remove_from_cart_count":0,"purchase_count":0})


In [12]:
# Now table have all windows, and purchases
windows_with_ga_1[windows_with_ga_1['CUSTOMER_ID'] == 501508595]


,CUSTOMER_ID,WINDOW_START_LOCAL,WINDOW_END_LOCAL,SALES_OFFICE,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC,TIMEZONE,add_to_cart_count,remove_from_cart_count,purchase_count,has_any_event,first_event_time,last_event_time
92578,501508595,2024-06-12 16:00:00,2024-06-19 16:00:00,G237,48 Hours,Wednesday,7,Sideload,America/Denver,0,0,0,False,2024-07-21 22:11:45.191,2025-05-25 09:53:07.531
92579,501508595,2024-06-19 16:00:00,2024-06-26 16:00:00,G237,48 Hours,Wednesday,7,Sideload,America/Denver,0,0,0,False,2024-07-21 22:11:45.191,2025-05-25 09:53:07.531
92580,501508595,2024-06-26 16:00:00,2024-07-03 16:00:00,G237,48 Hours,Wednesday,7,Sideload,America/Denver,0,0,0,False,2024-07-21 22:11:45.191,2025-05-25 09:53:07.531
92581,501508595,2024-07-03 16:00:00,2024-07-10 16:00:00,G237,48 Hours,Wednesday,7,Sideload,America/Denver,0,0,0,False,2024-07-21 22:11:45.191,2025-05-25 09:53:07.531
92582,501508595,2024-07-10 16:00:00,2024-07-17 16:00:00,G237,48 Hours,Wednesday,7,Sideload,America/Denver,0,0,0,False,2024-07-21 22:11:45.191,2025-05-25 09:53:07.531
92583,501508595,2024-07-17 16:00:00,2024-07-24 16:00:00,G237,48 Hours,Wednesday,7,Sideload,America/Denver,1,0,1,True,2024-07-21 22:11:45.191,2025-05-25 09:53:07.531
92584,501508595,2024-07-24 16:00:00,2024-07-31 16:00:00,G237,48 Hours,Wednesday,7,Sideload,America/Denver,1,1,1,True,2024-07-21 22:11:45.191,2025-05-25 09:53:07.531
92585,501508595,2024-07-31 16:00:00,2024-08-07 16:00:00,G237,48 Hours,Wednesday,7,Sideload,America/Denver,9,3,1,True,2024-07-21 22:11:45.191,2025-05-25 09:53:07.531
92586,501508595,2024-08-07 16:00:00,2024-08-14 16:00:00,G237,48 Hours,Wednesday,7,Sideload,America/Denver,1,0,1,True,2024-07-21 22:11:45.191,2025-05-25 09:53:07.531
92587,501508595,2024-08-14 16:00:00,2024-08-21 16:00:00,G237,48 Hours,Wednesday,7,Sideload,America/Denver,2,1,1,True,2024-07-21 22:11:45.191,2025-05-25 09:53:07.531


# Model, predict which window will have event

In [13]:
# Window cart activity rate
(windows_with_ga_1['add_to_cart_count'] > 0).mean()


0.1306506887174158

In [15]:
# Data prep (drop everything except add_to_cart_count for now)
windows_ga_activity = windows_with_ga_1.drop(
    columns=['remove_from_cart_count', 'purchase_count', 'has_any_event',
             'first_event_time', 'last_event_time']
)

# Create flag, then drop original column
windows_ga_activity['flag_add_to_cart'] = (windows_ga_activity['add_to_cart_count'] > 0).astype(int)
windows_ga_activity.drop(columns='add_to_cart_count', inplace=True)



In [19]:
windows_ga_activity.head()

,CUSTOMER_ID,WINDOW_START_LOCAL,WINDOW_END_LOCAL,SALES_OFFICE,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC,TIMEZONE,flag_add_to_cart
0,500245685,2024-06-10 17:00:00,2024-06-17 17:00:00,G111,48 Hours,Monday,7,OFS,America/Denver,0
1,500245685,2024-06-17 17:00:00,2024-06-24 17:00:00,G111,48 Hours,Monday,7,OFS,America/Denver,0
2,500245685,2024-06-24 17:00:00,2024-07-01 17:00:00,G111,48 Hours,Monday,7,OFS,America/Denver,0
3,500245685,2024-07-01 17:00:00,2024-07-08 17:00:00,G111,48 Hours,Monday,7,OFS,America/Denver,0
4,500245685,2024-07-08 17:00:00,2024-07-15 17:00:00,G111,48 Hours,Monday,7,OFS,America/Denver,0


In [22]:
# pip install imbalanced-learn
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, accuracy_score


resampler = SMOTE(k_neighbors=5, random_state=42)  # or RandomUnderSampler(0.5, random_state=42)

imb_pipe = ImbPipeline([
    ("prep", preproc),
    ("resample", resampler),
    ("gbt", gbt)
])

metrics = []
for train_idx, test_idx in gkf.split(X, y, groups=groups):
    X_tr = X.iloc[train_idx]; y_tr = y[train_idx]
    X_te = X.iloc[test_idx];  y_te = y[test_idx]

    imb_pipe.fit(X_tr, y_tr)   # resampling happens only on the training fold
    p = imb_pipe.predict_proba(X_te)[:, 1]

    metrics.append({
        "roc_auc": roc_auc_score(y_te, p),
        "pr_auc":  average_precision_score(y_te, p),
        "brier":   brier_score_loss(y_te, p, pos_label=1),
        "acc":     accuracy_score(y_te, (p>=0.5).astype(int))
    })

pd.DataFrame(metrics).mean().round(4)



KeyboardInterrupt: 